In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time

### Heuristic simplification
- When simplifying ZX-diagrams with T-spiders, simplification routines like full_reduce lead to a very high two-qubit gate count.
- When using heuristic-based approaches we can circumvent the problem to some extent leading to better overall circuit cost after optimization

In [2]:
random.seed(1344)
g = zx.generate.cliffordT(qubits=8, depth=1000, p_t=0.4)
c = zx.Circuit.from_graph(g)
print(c.stats())

Circuit  on 8 qubits with 1000 gates.
        401 is the T-count
        599 Cliffords among which 
        207 2-qubit gates (207 CNOT, 0 other) and
        0 Hadamard gates.


In [3]:
c = zx.optimize.basic_optimization(c.split_phase_gates())
g = c.to_graph()
g_tele = zx.simplify.teleport_reduce(g)
g_tele.track_phases = False
zx.Circuit.from_graph(g).split_phase_gates().stats()

'Circuit  on 8 qubits with 612 gates.\n        137 is the T-count\n        475 Cliffords among which \n        201 2-qubit gates (191 CNOT, 10 other) and\n        123 Hadamard gates.'

In [4]:
g_full = g_tele.copy()
zx.simplify.full_reduce(g_full)
print(zx.extract_circuit(g_full.copy()).stats())
g_full.normalize()
zx.draw(g_full)

Circuit  on 8 qubits with 735 gates.
        135 is the T-count
        600 Cliffords among which 
        419 2-qubit gates (74 CNOT, 345 other) and
        180 Hadamard gates.


In [5]:
g_greedy = g_tele.copy()
zx.simplify.greedy_simp(g_greedy)
print(zx.extract_circuit(g_greedy.copy()).stats())
g_greedy.normalize()
zx.draw(g_greedy)

Circuit  on 8 qubits with 704 gates.
        135 is the T-count
        569 Cliffords among which 
        194 2-qubit gates (1 CNOT, 193 other) and
        292 Hadamard gates.


In [6]:
g_nu = g_tele.copy()
zx.simplify.greedy_simp_neighbors(g_nu)
zx.extract_circuit(g_nu.copy(), quiet=False).stats()

Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 2
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 6
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 2
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 2
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 5
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 2
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 2
Simple vertex
Vertices extra

'Circuit  on 8 qubits with 698 gates.\n        135 is the T-count\n        563 Cliffords among which \n        190 2-qubit gates (0 CNOT, 190 other) and\n        290 Hadamard gates.'

In [7]:
g_nu.normalize()
print(g_nu.stats())
zx.draw(g_nu)

Graph(307 vertices, 486 edges)
degree distribution: 
1: 16
2: 61
3: 132
4: 66
5: 24
6: 4
7: 3
9: 1



In [8]:
g_nu_sim = g_tele.copy()
g_nu_sim = zx.simplify.sim_anneal_simp_neighbors(g_nu_sim, iterations=100, alpha=0.99, cap=-50)


start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch

reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to best eval
start branch with negative match
reset to bes

In [9]:
print(zx.extract_circuit(g_nu_sim.copy(), quiet=False).stats())
g_norm = g_nu_sim.copy()
g_norm.normalize()
print(g_norm.stats())
zx.draw(g_norm)

Simple vertex
Vertices extracted: 6
Simple vertex
Vertices extracted: 5
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 5
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 6
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 2
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 2
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 5
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 3
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 2
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 4
Simple vertex
Vertices extracted: 2
Simple vertex
Vertices extra

In [9]:
g_norm = g_nu_sim.copy()
g_norm.normalize()

In [10]:
zx.draw(g_norm)

In [10]:
zx.compare_tensors(zx.extract_circuit(g_nu_sim.copy()),c)

True